# 이미지 분류 정확도 개선

In [33]:
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization, Activation
from  keras.callbacks import ModelCheckpoint
from keras import regularizers, optimizers
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

## 데이터 준비

In [8]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [10]:
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

(x_train, x_valid) = x_train[5000:], x_train[:5000]
(y_train, y_valid) = y_train[5000:], y_train[:5000]

In [12]:
print(x_train.shape)
print(x_valid.shape)
print(x_test.shape)

(45000, 32, 32, 3)
(5000, 32, 32, 3)
(10000, 32, 32, 3)


In [24]:
mean = np.mean(x_train, axis = (0,1,2,3))
std = np.std(x_train, axis = (0,1,2,3))

x_train = (x_train - mean) / (std )
x_valid = (x_valid - mean) / (std) 
x_test = (x_test - mean) / (std)

In [28]:
num_classes = 10

y_train = to_categorical(y_train, num_classes)
y_valid = to_categorical(y_valid, num_classes)
y_test = to_categorical(y_test, num_classes)

In [29]:
datagen = ImageDataGenerator(
        rotation_range = 15,
        width_shift_range = 0.1,
        height_shift_range = 0.1,
        horizontal_flip = True,
        vertical_flip = False
)

datagen.fit(x_train)

## 모델 정의하기

In [36]:
base_hidden_units = 32
weight_decay = 1e-4
INPUT_SHAPE = x_train.shape[1:]
model = Sequential()

model.add(Conv2D(base_hidden_units, kernel_size = 3, padding = 'same', 
                 kernel_regularizer = regularizers.l2(weight_decay), input_shape = INPUT_SHAPE))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv2D(base_hidden_units, kernel_size = 3, padding = 'same', 
                 kernel_regularizer = regularizers.l2(weight_decay), input_shape = INPUT_SHAPE))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(base_hidden_units * 2, kernel_size = 3, padding = 'same', 
                 kernel_regularizer = regularizers.l2(weight_decay), input_shape = INPUT_SHAPE))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv2D(base_hidden_units * 2, kernel_size = 3, padding = 'same', 
                 kernel_regularizer = regularizers.l2(weight_decay), input_shape = INPUT_SHAPE))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(base_hidden_units * 4, kernel_size = 3, padding = 'same', 
                 kernel_regularizer = regularizers.l2(weight_decay), input_shape = INPUT_SHAPE))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv2D(base_hidden_units * 4, kernel_size = 3, padding = 'same', 
                 kernel_regularizer = regularizers.l2(weight_decay), input_shape = INPUT_SHAPE))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(10, activation = 'softmax'))

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        896       
                                                                 
 activation (Activation)     (None, 32, 32, 32)        0         
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 32, 32, 32)        0         
                                                                 
 batch_normalization_1 (Batc  (None, 32, 32, 32)       128       
 hNormalization)                                        